In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [8]:
# pip install accelerate
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM


# quantization_config = BitsAndBytesConfig(load_in_8bit=True)

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-2b",
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=32)
print(tokenizer.decode(outputs[0]))

<bos>Write me a poem about Machine Learning.

[Answer 1]

<strong>Machine Learning</strong>

Machine learning is a field of study that is very much like a machine.

It is a field


# Use LangChain

In [12]:
from langchain.llms import HuggingFacePipeline
from transformers import pipeline


pipe = pipeline(
    "text-generation",
    # "question-answering",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=100,
)
local_llm = HuggingFacePipeline(pipeline=pipe)

The model 'Gemma2ForCausalLM' is not supported for question-answering. Supported models are ['AlbertForQuestionAnswering', 'BartForQuestionAnswering', 'BertForQuestionAnswering', 'BigBirdForQuestionAnswering', 'BigBirdPegasusForQuestionAnswering', 'BloomForQuestionAnswering', 'CamembertForQuestionAnswering', 'CanineForQuestionAnswering', 'ConvBertForQuestionAnswering', 'Data2VecTextForQuestionAnswering', 'DebertaForQuestionAnswering', 'DebertaV2ForQuestionAnswering', 'DistilBertForQuestionAnswering', 'ElectraForQuestionAnswering', 'ErnieForQuestionAnswering', 'ErnieMForQuestionAnswering', 'FalconForQuestionAnswering', 'FlaubertForQuestionAnsweringSimple', 'FNetForQuestionAnswering', 'FunnelForQuestionAnswering', 'GPT2ForQuestionAnswering', 'GPTNeoForQuestionAnswering', 'GPTNeoXForQuestionAnswering', 'GPTJForQuestionAnswering', 'IBertForQuestionAnswering', 'LayoutLMv2ForQuestionAnswering', 'LayoutLMv3ForQuestionAnswering', 'LEDForQuestionAnswering', 'LiltForQuestionAnswering', 'LlamaFor

In [11]:
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema import StrOutputParser


base_template = """
<bos><start_of_turn>user
{question}
<end_of_turn><start_of_turn>model
"""

qa_chain = (
    {"question": RunnablePassthrough()} | PromptTemplate.from_template(base_template) | local_llm | StrOutputParser()
)
res = qa_chain.invoke("日本で一番高い山の名前と標高は何ですか？")
print(res)

日本には、標高が1000mを超える山が約1000本あります。
日本には、標高が1000mを超える山が約1000本あります。
日本には、標高が1000mを超える山が約1000本あります。
日本には、標高が1000mを超える山が約1000本あります。
日本には、標
